## Dolly local inference 테스트

### Dolly 모델 주소
- dolly-v2-7b : https://huggingface.co/databricks/dolly-v2-7b
- dolly-v2-12b : https://huggingface.co/databricks/dolly-v2-12b

### Local mode inference
- 일반적인 Jupyter notebook을 사용하는 것과 동일하게 사용할 수 있습니다.

### Tested version

Tested on `Python 3.9.15`

```
sagemaker: 2.146.0
transformers: 4.29.2
torch: 1.13.1
accelerate: 0.19.0
```


In [ ]:
!pip install -q transformers accelerate

In [ ]:
import sagemaker
import transformers
import torch
import accelerate
print(sagemaker.__version__)
print(transformers.__version__)
print(torch.__version__)
print(accelerate.__version__)

In [ ]:
sagemaker_session = sagemaker.Session()

### 모델 다운로드

- 로컬에서 테스트를 하기 위해 모델을 다운로드 받습니다. git lfs 명령어를 사용해도 되지만 여기서는 huggingface 에서 제공하는 라이브러리를 사용하였습니다.
- git lfs 사용 시 아래와 같이 받을 수 있습니다.
```
git lfs install
git clone https://huggingface.co/databricks/dolly-v2-7b
```


In [ ]:
from huggingface_hub import snapshot_download
from pathlib import Path
import os

local_model_path = Path("./pretrained-models")
local_model_path.mkdir(exist_ok=True)
model_name = "databricks/dolly-v2-7b"
allow_patterns = ["*.json", "*.pt", "*.bin", "*.txt", "*.model"]

model_download_path = snapshot_download(
    repo_id=model_name,
    cache_dir=local_model_path,
    allow_patterns=allow_patterns,
)

### 나중에 사용하기 위해 모델 업로드

- 나중에 활용하기 위해 s3에 모델을 업로드 해놓도록 합니다.
- huggingface 라이브러리를 사용한 경우에는 상관 없지만, git lfs 를 사용해 다운로드 받은 경우에는 `.git` 디렉토리를 삭제하여 쓸모없는 업로드를 하지 않도록 해야 합니다.

### instruct_pipeline.py 다운로드
- 실제 생성을 할 때의 pipeline 스크립트는 [여기](https://huggingface.co/databricks/dolly-v2-7b/blob/main/instruct_pipeline.py) 있습니다. 해당 스크립트는 모델 로딩 시 `trust_remote_code=True` 로 설정하면 필요 없지만, 이걸 내장하거나 추후 수정해서 사용할 수 있으므로 현재 디렉토리에 다운받아 놓도록 합니다. 예를 들어 아래와 같이 다운로드 받을 수 있습니다.
```
wget https://huggingface.co/databricks/dolly-v2-7b/raw/main/instruct_pipeline.py
```

In [ ]:
print(f"Model downloaded: {model_download_path}")
s3_model_prefix = "llm/databricks/dolly-v2-7b/model"  # folder where model checkpoint will go
model_artifact = sagemaker_session.upload_data(path=model_download_path, key_prefix=s3_model_prefix)
print(f"s3 Model uploaded: {model_artifact}")

In [ ]:
import os
from instruct_pipeline import InstructionTextGenerationPipeline
from transformers import AutoModelForCausalLM, AutoTokenizer

model_location = model_download_path

# 특정한 위치에 모델 파일들을 넣어놓는 경우
# model_location = "./dolly-imdb-finetune"

tokenizer = AutoTokenizer.from_pretrained(model_location, padding_side="left")
model = AutoModelForCausalLM.from_pretrained(model_location, device_map="auto", torch_dtype=torch.bfloat16)


In [ ]:
generate_text = InstructionTextGenerationPipeline(
    model = model,
    tokenizer = tokenizer,
    do_sample = True,
    max_new_tokens = 256
)

### Inference 테스트

- 모델이 성공적으로 로딩되면 테스트를 진행할 수 있습니다.

In [ ]:
prompt = "Explain to me how to use aws serverless services"

In [ ]:
print(prompt)

In [ ]:
%%time
res = generate_text(prompt)

In [ ]:
print(res[0]["generated_text"])

### 결과
- 텍스트가 잘 생성되는 것을 확인할 수 있습니다. Prompt tuning을 통해 더 좋은 결과를 얻을 수 있습니다.
- 모델이 업로드된 s3 주소는 추후 사용을 위해서 메모해 놓도록 합니다.

In [ ]:
%store model_artifact
%store model_download_path